In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt

%matplotlib inline
%run load_data.ipynb

In [2]:
word_count = get_word_counts() #TAKE OUT STORE DATA IN FUNCTIONS IN CASE CHANGES OF DATABASE
stock_prices = get_stock_prices()

word_data = pd.DataFrame(data=word_count, columns=['id', 'word', 'count', 'date'])
stock_data = pd.DataFrame(data=stock_prices, columns=['id', 'openprice', 'closeprice', 'date'])

Excluding: None	None	None	2018-03-07


In [3]:
word_data.head()

,id,word,count,date
0,1,birthday,1,2018-03-07
1,2,httpstco8taibf6hls,1,2018-03-07
2,3,production,1,2018-03-07
3,4,tesla,2,2018-03-07
4,5,doom,1,2018-03-07


In [4]:
stock_data.head()

,id,openprice,closeprice,date
0,1,253.05,249.24,2017-02-01
1,2,248.34,251.55,2017-02-02
2,3,251.91,251.33,2017-02-03
3,4,251.00,257.77,2017-02-06
4,5,258.19,257.48,2017-02-07


In [5]:
'''
print("Max stock data: {}".format(np.max(stock_data['date'])))
print("Max tweet date: {}".format(np.max(word_data['date'])))
'''

'\nprint("Max stock data: {}".format(np.max(stock_data[\'date\'])))\nprint("Max tweet date: {}".format(np.max(word_data[\'date\'])))\n'

### Accumulate tweets from weekends and holidays to the next open stock market day

In [6]:
'''
last_stock_date = np.max(stock_data['date'])
word_data.drop(labels=word_data.loc[word_data['date'] > last_stock_date], axis=1)
'''
while(pd.unique(word_data['date'].isin(stock_data['date'])).shape[0] > 1):
    word_data.loc[~word_data['date'].isin(stock_data['date']), 'date'] += dt.timedelta(days=1) 


### Getting Percentage stock increase by word count

In [8]:
agg = pd.merge(stock_data, word_data, how='right', on='date', sort=True, suffixes=('_stock', '_word'))

agg.head()

,id_stock,openprice,closeprice,date,id_word,word,count
0,14,275.45,277.39,2017-02-21,7885,no,1
1,14,275.45,277.39,2017-02-21,7886,rt,2
2,14,275.45,277.39,2017-02-21,7887,been,1
3,14,275.45,277.39,2017-02-21,7888,httpstco6ve3yjostm,1
4,14,275.45,277.39,2017-02-21,7889,falcon,1


## Building word_stats table:

In [9]:
word_stats = pd.DataFrame(pd.unique(agg['word']), columns=['word'])
word_stats['count'] = word_stats['word'].apply(lambda x: np.sum(agg.loc[agg['word'] == x, 'count']))

In [10]:
total_words = np.sum(word_stats['count'])
word_stats['prob_occur'] = word_stats['count'].apply(lambda x: x / total_words * 100)

word_stats['prob_inc'] = word_stats['word'].apply(lambda x: np.sum(agg.loc[(agg['word'] == x) & (agg['closeprice'] - agg['openprice'] >= 0), 'count']))
word_stats['prob_inc'] = word_stats.apply(lambda x: x['prob_inc'] / x['count'], axis=1)

word_stats['prob_dec'] = word_stats.apply(lambda x: 1.0 - x['prob_inc'], axis = 1)
word_stats.head()

,word,count,prob_occur,prob_inc,prob_dec
0,no,101,0.263261,0.534653,0.465347
1,rt,320,0.834094,0.496875,0.503125
2,been,34,0.088622,0.617647,0.382353
3,httpstco6ve3yjostm,1,0.002607,1.000000,0.000000
4,falcon,97,0.252835,0.453608,0.546392


In [11]:
word_stats[['word', 'prob_inc', 'prob_occur']].to_csv('word_stats.csv')

# PREDICTIONS:
## p_inc(W) = (p_inc(w_1) + p_inc(w_2) + p_inc(w_3) + ... + p_inc(w_n)) / n

#### Average of probabilities of increase for each word spoken in tweets prior to stock day.

In [12]:
stock_data['num_words'] = stock_data.apply(lambda x: agg.loc[agg['date'] == x['date'], 'count'].sum(), axis=1)

agg = agg.merge(word_stats[['word', 'prob_inc']], how='left', on='word')

In [13]:
agg.head()

,id_stock,openprice,closeprice,date,id_word,word,count,prob_inc
0,14,275.45,277.39,2017-02-21,7885,no,1,0.534653
1,14,275.45,277.39,2017-02-21,7886,rt,2,0.496875
2,14,275.45,277.39,2017-02-21,7887,been,1,0.617647
3,14,275.45,277.39,2017-02-21,7888,httpstco6ve3yjostm,1,1.000000
4,14,275.45,277.39,2017-02-21,7889,falcon,1,0.453608


In [14]:
stock_data.head()

,id,openprice,closeprice,date,num_words
0,1,253.05,249.24,2017-02-01,0
1,2,248.34,251.55,2017-02-02,0
2,3,251.91,251.33,2017-02-03,0
3,4,251.00,257.77,2017-02-06,0
4,5,258.19,257.48,2017-02-07,0


In [16]:
agg['product'] = agg.apply(lambda x: x['count'] * x['prob_inc'], axis=1)

In [17]:
agg.head()

,id_stock,openprice,closeprice,date,id_word,word,count,prob_inc,product
0,14,275.45,277.39,2017-02-21,7885,no,1,0.534653,0.534653
1,14,275.45,277.39,2017-02-21,7886,rt,2,0.496875,0.993750
2,14,275.45,277.39,2017-02-21,7887,been,1,0.617647,0.617647
3,14,275.45,277.39,2017-02-21,7888,httpstco6ve3yjostm,1,1.000000,1.000000
4,14,275.45,277.39,2017-02-21,7889,falcon,1,0.453608,0.453608


In [18]:
stock_data['prediction'] = stock_data.apply(lambda x: agg.loc[x['date'] == agg['date'], 'product'].sum(), axis=1)
stock_data.head()

,id,openprice,closeprice,date,num_words,prediction
0,1,253.05,249.24,2017-02-01,0,0.0
1,2,248.34,251.55,2017-02-02,0,0.0
2,3,251.91,251.33,2017-02-03,0,0.0
3,4,251.00,257.77,2017-02-06,0,0.0
4,5,258.19,257.48,2017-02-07,0,0.0


In [19]:
stock_data['prediction'] = stock_data.apply(lambda x: x['prediction'] / x['num_words'] if (x['num_words'] > 0) else 0, axis=1)

In [20]:
stock_data.head()

,id,openprice,closeprice,date,num_words,prediction
0,1,253.05,249.24,2017-02-01,0,0.0
1,2,248.34,251.55,2017-02-02,0,0.0
2,3,251.91,251.33,2017-02-03,0,0.0
3,4,251.00,257.77,2017-02-06,0,0.0
4,5,258.19,257.48,2017-02-07,0,0.0


In [21]:
stock_data['direction'] = np.zeros((stock_data.shape[0]))
stock_data['direction'] = stock_data.apply(lambda x: 1 if x['prediction'] > 0.5 else (-1 if x['prediction'] < 0.5 else 0), axis=1)
stock_data['direction'] = stock_data.apply(lambda x: 0 if x['num_words'] == 0 else x['num_words'], axis=1)

In [22]:
stock_data.head()

,id,openprice,closeprice,date,num_words,prediction,direction
0,1,253.05,249.24,2017-02-01,0,0.0,0
1,2,248.34,251.55,2017-02-02,0,0.0,0
2,3,251.91,251.33,2017-02-03,0,0.0,0
3,4,251.00,257.77,2017-02-06,0,0.0,0
4,5,258.19,257.48,2017-02-07,0,0.0,0


In [23]:
stock_data[['date','direction']].to_csv("predictions.csv")